In [1]:
import json
import pandas as pd
import os
import sys
import pickle
import numpy as np
import math
import random
import math
import torch
import torchvision 
import torch.nn.functional as F  
import torchvision.datasets as datasets  
import torchvision.transforms as transforms  
from torch import optim  
from torch import nn  
from torch.utils.data import DataLoader  
from tqdm import tqdm  
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [2]:
# parentPath = '/content/drive/MyDrive/PhD/Fragle_TSS/Tested_Algorithms/Large Bin Based Modeling/Dataset/Sig_10_Mb_10_Splits'
# os.chdir(parentPath)

# partition_no = 15 # final
# uniform_epochs = 0
# non_uniform_epoch_no = 100 # final 
# non_uniform_schedule_width = 5 # final

In [3]:
parentPath = '/content/drive/MyDrive/PhD/Fragle_TSS/Tested_Algorithms/Large Bin Based Modeling/Dataset/Sig_10_Mb_10_Splits'
os.chdir(parentPath)

partition_no = 15 
uniform_epochs = 70
non_uniform_epoch_no = 30  
non_uniform_schedule_width = 5

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model, criterion, optimizer = None, None, None
train_indices, test_indices, train_partition_indices, train_y = [], [], [], []
train_samples, test_samples, max_arr = None, None, None
train_meta_info, test_meta_info = None, None

json_file = open('../../meta_info_files/split_patient_wise.json')
dic = json.load(json_file)
myPath = os.getcwd()

thresholds = [0.05, 0.03, 0.02, 0.01, 0.005, 0.001]
MAE_dic, SN_dic, SP_dic = {}, {}, {}
for thr in thresholds:
    MAE_dic[thr] = []
    SN_dic[thr] = []
    SP_dic[thr] = []

In [5]:
loaded_dict = {}
with open('train_samples.pkl', 'rb') as f:
  loaded_dict = pickle.load(f)
train_meta_info = loaded_dict['meta'] 
train_samples = loaded_dict['samples']

train_y = []
for info in train_meta_info:
  train_y.append(info[-1])
train_y = np.array(train_y)

loaded_dict = {}
with open('test_samples.pkl', 'rb') as f:
  loaded_dict = pickle.load(f)
test_meta_info = loaded_dict['meta'] 
test_samples = loaded_dict['samples']
loaded_dict = {}

In [6]:
sums = np.sum(train_samples, axis=2)
train_samples = train_samples/ sums[:, :, np.newaxis]

sums = np.sum(test_samples, axis=2)
test_samples = test_samples/ sums[:, :, np.newaxis]

In [7]:
class LoadDataset(Dataset):
    def __init__(self, indices, train):
        self.indices = indices
        self.is_train = train

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, index):
        ind = self.indices[index]
        if self.is_train == True:
          dataY = torch.tensor(train_meta_info[ind][-1])
          dataX = torch.tensor(train_samples[ind]/ max_arr)
        else:
          dataY = torch.tensor(test_meta_info[ind][-1])
          dataX = torch.tensor(test_samples[ind]/ max_arr)
        dataX = dataX.float()
        dataY = dataY.float()
        return (dataX, dataY)

In [8]:
# tmp = torch.tensor([5.0])
# print(len(tmp.size()))

In [9]:
class VGG_convnet(nn.Module):

    def __init__(self, bin_no = 282, feature_no = 156):

        super(VGG_convnet, self).__init__()

        self.begin_linear1 = nn.Linear(feature_no, 64)
        self.begin_linear1_rw = torch.nn.Parameter(torch.randn(64))
        self.begin_linear2 = nn.Linear(64, 96)
        self.begin_linear2_rw = torch.nn.Parameter(torch.randn(96))
        self.begin_linear3 = nn.Linear(96, 128)
        self.begin_linear3_rw = torch.nn.Parameter(torch.randn(128))
        
        self.pos_emb1D = torch.nn.Parameter(torch.randn(bin_no, 128))
        self.const = torch.nn.Parameter(torch.tensor(0.01))
        
        # block 1:         
        self.conv1a = nn.Conv1d(in_channels=128, out_channels=32, kernel_size=3, padding=1)
        self.conv1b = nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3, padding=1)
        self.LN1 = nn.LayerNorm(bin_no)
        
        # block 2:      
        self.conv2a = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv2b = nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        self.LN2 = nn.LayerNorm(bin_no)
        
        # block 3:            
        self.conv3a =  nn.Conv1d(in_channels=64, out_channels=96, kernel_size=3, padding=1)
        self.conv3b =  nn.Conv1d(in_channels=96, out_channels=96, kernel_size=3, padding=1)
        self.LN3 = nn.LayerNorm(bin_no)
        
        #block 4:       
        self.conv4a = nn.Conv1d(in_channels=96, out_channels=128, kernel_size=3, padding=1)
        self.conv4b = nn.Conv1d(in_channels=128, out_channels=128, kernel_size=3, padding=1)
        self.LN4 = nn.LayerNorm(bin_no)
        
        # linear layers:
        self.end_linear1 = nn.Linear(128, 128)
        self.end_linear1_rw = torch.nn.Parameter(torch.randn(128))
        self.end_linear2 = nn.Linear(128, 64)
        self.end_linear2_rw = torch.nn.Parameter(torch.randn(64))
        self.end_linear3 = nn.Linear(64, 1)

    def forward(self, x, targets, isTrain):
        x = self.begin_linear1(x)
        x = F.relu(x) + self.begin_linear1_rw * x * F.relu(x)
        x = self.begin_linear2(x)
        x = F.relu(x) + self.begin_linear2_rw * x * F.relu(x)
        x = self.begin_linear3(x)
        x = F.relu(x) + self.begin_linear3_rw * x * F.relu(x)

        x = x + self.pos_emb1D
        x = x.permute(0, 2, 1) # 128, 282
        
        # block 1:  
        x = F.relu(self.conv1a(x)) # 32, 282
        x = F.relu(self.conv1b(x))
        x = self.LN1(x)
        
        # block 2:   
        x = F.relu(self.conv2a(x)) # 64, 282
        x = F.relu(self.conv2b(x))
        x = self.LN2(x)

        # block 3:    
        x = F.relu(self.conv3a(x)) # 96, 282
        x = F.relu(self.conv3b(x))
        x = self.LN3(x)
        
        #block 4:      
        x = F.relu(self.conv4a(x)) # 128, 282
        x = F.relu(self.conv4b(x))
        x = self.LN4(x)
        
        y = torch.mean(x, axis=2) # 128
        
        # linear layers:   
        y = self.end_linear1(y)
        y = F.relu(y) + self.end_linear1_rw * y * F.relu(y)
        y = self.end_linear2(y)
        y = F.relu(y) + self.end_linear2_rw * y * F.relu(y)
        y = self.end_linear3(y)
        scores = y.squeeze()

        final_loss = None
        if isTrain == True:
          numerator = self.const + torch.abs(scores - targets)
          denominator = self.const + targets
          loss = None
          if len(scores.size())==0:
            loss = torch.sum(numerator/denominator)
          else:
            loss = torch.sum(numerator/denominator)/ scores.size(0)
          final_loss = loss * loss
        
        return y, final_loss

In [10]:
def metric_calc(loader, thr):
    running_MAE = 0
    num_batches = 0
    TP, TN, FP, FN = 0, 0, 0, 0
    model.eval()
    with torch.no_grad():
        for batch_idx, (data, targets) in enumerate(loader):
            # Get data to cuda if possible
            data = data.to(device=device)
            targets = targets.to(device=device)

            # forward
            scores, _ = model(data, None, False)
            scores = scores.view(targets.size())
            
            # metric calc (MAE)
            #MAE_batch = L1criterion(scores, targets)
            MAE_batch = criterion(scores, targets)
            running_MAE = running_MAE + MAE_batch.item()
            
            # TP, TN, FP, FN
            for i in range(targets.size(0)):
                # if targets[i].item()>0.0 and targets[i].item()<0.01:
                if targets[i].item()>0.0 and targets[i].item()<thr:
                     continue
                elif scores[i].item()>thr and targets[i].item()>0.0:
                    TP += 1
                elif scores[i].item()<=thr and targets[i].item()>0.0:
                    FN+=1
                elif scores[i].item()>thr and targets[i].item()==0.0:
                    FP+=1
                elif scores[i].item()<=thr and targets[i].item()==0.0:
                    TN+=1

            num_batches+=1
    
    MAE_final = running_MAE/num_batches
    SN_final = TP/(TP+FN)
    SP_final = TN/(TN+FP)
    
    return MAE_final, SN_final, SP_final

In [11]:
def make_csv(split_no, max_arr):
    csv_list = []
    test_y = []
    scores, sample_names = [], []
    
    model.eval()
    with torch.no_grad():
      for index in test_indices:
          sample_names.append(test_meta_info[index][1])
          dataX = np.copy(test_samples[index])
          dataX = dataX/ max_arr
          test_y.append(test_meta_info[index][-1])

          dataX = torch.tensor(dataX)
          # dataX = dataX.permute(1, 0)     ########
          dataX = torch.unsqueeze(dataX, dim=0)
          dataX = dataX.float()

          score, _ = model(dataX.to(device), None, False)
          scores.append(score.item())
        
    for i in range(len(scores)):
        tmp = []
        tmp.append(sample_names[i])
        tmp.append(scores[i])
        tmp.append(test_y[i])
        csv_list.append(tmp)
        
    folder = '/content/drive/MyDrive/PhD/Fragle_TSS/Tested_Algorithms/Large Bin Based Modeling/10_split_csv/'
    filePath = folder + 'test' + str(split_no) + '.csv'
    my_df = pd.DataFrame(csv_list)
    my_df.to_csv(filePath, index=False, header=['Sample_ID', 'Pred_Fraction', 'True_Fraction'])

In [12]:
def get_partition_probabilities(train_partition_indices):
  partition_losses = []
  model.eval()
  with torch.no_grad():
    for partition in train_partition_indices:
      dataX = train_samples[partition, :, :] 
      dataX = dataX/ max_arr
      dataX = torch.tensor(dataX).float()
      dataY = []
      for i in range(len(partition)):
        ind = partition[i]
        dataY.append(train_y[ind])
      dataY = torch.tensor(dataY).float()

      _, loss = model(dataX.to(device), dataY.to(device), True)
      partition_losses.append(loss.item())

  parititon_probabilities = partition_losses/ np.sum(partition_losses)
  return parititon_probabilities

In [13]:
# total 140 samples
# 3 partitions -> 50, 50, 40
# Equal probability -> 1/3 = 0.3333
# 3 probabilities -> 0.5, 0.4, 0.1
# sample no from partition 1: 50 X (0.5/ 0.3333), partition 2: 50 X (0.4/ 0.3333) and partition 3: 40 X (0.1/ 0.3333)

In [14]:
def sample_train_indices(train_partition_indices, partition_probabilities):
  chosen_indices = []
  uniform_probability = float(1/ len(train_partition_indices))
  for i in range(len(train_partition_indices)):
    sample_no = round(len(train_partition_indices[i]) * (partition_probabilities[i]/ uniform_probability))
    chosen_indices.extend( random.choices(train_partition_indices[i], k=sample_no) )
  return chosen_indices

In [15]:
def get_train_indices(uniform_sample_ratio, train_partition_indices):
  non_uniform_sample_ratio = 1 - uniform_sample_ratio
  train_indices = []
  uniform_prob = float(1/ len(train_partition_indices)) * uniform_sample_ratio
  uniform_probabilities = []
  for i in range(len(train_partition_indices)):
    uniform_probabilities.append(uniform_prob)
  non_uniform_probabilities = get_partition_probabilities(train_partition_indices)
  for i in range(len(non_uniform_probabilities)):
    non_uniform_probabilities[i] = non_uniform_probabilities[i] * non_uniform_sample_ratio
  
  train_indices.extend(sample_train_indices(train_partition_indices, uniform_probabilities))
  train_indices.extend(sample_train_indices(train_partition_indices, non_uniform_probabilities))
  return train_indices

In [16]:
def train(train_indices, train_partition_indices, test_loader):
  train_data = LoadDataset(train_indices, True)
  train_loader = DataLoader(dataset=train_data, batch_size=32, shuffle=True)
  for epoch in range(uniform_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device=device)
        data.requires_grad_()
        targets = targets.to(device=device)
        scores, loss = model(data, targets, True)
        scores = scores.view(targets.size())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # if epoch % 5 == 0:
    #   print(f'uniform epoch {epoch}')
    #   for thr in thresholds:
    #     MAE, SN, SP = metric_calc(test_loader, thr)
    #     print(f'threshold: {thr*100}%, MAE: {MAE}, Sensitivity: {SN}, Specificity: {SP}')

  T = non_uniform_schedule_width
  model.const.requires_grad = False
  # print(model.const.item())
  for epoch in range(non_uniform_epoch_no):
    Tc = epoch % T
    uniform_sample_ratio = 0.5 * (1 + math.cos(math.pi * (Tc/ T))) # turn this variable to 0 to stop uniform sampling
    # uniform_sample_ratio = 0
    train_indices = get_train_indices(uniform_sample_ratio, train_partition_indices)
    train_data = LoadDataset(train_indices, True)
    train_loader = DataLoader(dataset=train_data, batch_size=32, shuffle=True)
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device=device)
        data.requires_grad_()
        targets = targets.to(device=device)
        scores, loss = model(data, targets, True)
        scores = scores.view(targets.size())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # if (epoch+1) % 5 == 0:
    #   print(f'non-uniform epoch {epoch}')
    #   print(model.const.item())
    #   for thr in thresholds:
    #     MAE, SN, SP = metric_calc(test_loader, thr)
    #     print(f'threshold: {thr*100}%, MAE: {MAE}, Sensitivity: {SN}, Specificity: {SP}')

In [17]:
for N in range(10):
  split_train = dic['train'][N] + dic['val'][N]
  split_test = dic['test'][N]
  train_indices, test_indices, train_indices_output = [], [], []
  max_arr = []

  for i in range(len(train_meta_info)):
    if train_meta_info[i][1] in split_train:
      train_indices.append(i)
      train_indices_output.append((i, train_meta_info[i][-1]))
  train_indices_output = sorted(train_indices_output, key = lambda x : x[1])
  train_indices_sorted = []
  for item in train_indices_output:
    train_indices_sorted.append(item[0])
  partition_step = math.ceil(len(train_indices_output)/ partition_no)
  train_partitions, train_partition_indices = [], []
  for i in range(0, len(train_indices_output), partition_step):
    train_partitions.append([i, min(i+partition_step, len(train_indices_output))])
  for partition in train_partitions:
    train_partition_indices.append(train_indices_sorted[partition[0]: partition[1]])

  for i in range(len(test_meta_info)):   
    if test_meta_info[i][1] in split_test:
      test_indices.append(i)
  max_arr = np.max(train_samples[train_indices, :,:], axis=0)

  test_data = LoadDataset(test_indices, False)
  test_loader = DataLoader(dataset=test_data, batch_size=32)

  # initializing model for split N
  model = VGG_convnet().to(device)
  criterion = nn.L1Loss()
  optimizer = optim.Adam(model.parameters(), lr=0.0001)

  # train the model
  print(f'Split no. {N}')
  train(train_indices, train_partition_indices, test_loader) 

  # # model_path = '../../Experiments/Models/model' + str(N) + '.pt'
  # # torch.save(model.state_dict(), model_path)
  make_csv(N+1, max_arr)

  for thr in thresholds:
      MAE, SN, SP = metric_calc(test_loader, thr)
      print(f'threshold: {thr*100}%, MAE: {MAE}, Sensitivity: {SN}, Specificity: {SP}')
      MAE_dic[thr].append(MAE)
      SN_dic[thr].append(SN)
      SP_dic[thr].append(SP)
  print()
  print()
  

Split no. 0
threshold: 5.0%, MAE: 0.03749666536565532, Sensitivity: 0.8590604026845637, Specificity: 1.0
threshold: 3.0%, MAE: 0.03749666536565532, Sensitivity: 0.8605263157894737, Specificity: 1.0
threshold: 2.0%, MAE: 0.03749666536565532, Sensitivity: 0.8525345622119815, Specificity: 0.9629629629629629
threshold: 1.0%, MAE: 0.03749666536565532, Sensitivity: 0.860655737704918, Specificity: 0.9629629629629629
threshold: 0.5%, MAE: 0.03749666536565532, Sensitivity: 0.8679611650485437, Specificity: 0.9259259259259259
threshold: 0.1%, MAE: 0.03749666536565532, Sensitivity: 0.8347978910369068, Specificity: 0.8148148148148148


Split no. 1
threshold: 5.0%, MAE: 0.041385510597716675, Sensitivity: 0.7748344370860927, Specificity: 1.0
threshold: 3.0%, MAE: 0.041385510597716675, Sensitivity: 0.7881136950904393, Specificity: 1.0
threshold: 2.0%, MAE: 0.041385510597716675, Sensitivity: 0.8058690744920993, Specificity: 0.9629629629629629
threshold: 1.0%, MAE: 0.041385510597716675, Sensitivity: 0.8

In [18]:
for thr in thresholds:
    print(f'At threshold of {thr*100}%')
    mean_MAE = round(np.mean(MAE_dic[thr]), 3)
    med_MAE = round(np.median(MAE_dic[thr]), 3)
    std_MAE = round(np.std(MAE_dic[thr]), 3)
    print(f'MAE mean: {mean_MAE}, median: {med_MAE}, std: {std_MAE}')
        
    mean_SN = round(np.mean(SN_dic[thr]), 3)
    med_SN = round(np.median(SN_dic[thr]), 3)
    std_SN = round(np.std(SN_dic[thr]), 3)
    print(f'Sensitivity mean: {mean_SN}, median: {med_SN}, std: {std_SN}')
    
    mean_SP = round(np.mean(SP_dic[thr]), 3)
    med_SP = round(np.median(SP_dic[thr]), 3)
    std_SP = round(np.std(SP_dic[thr]), 3)
    print(f'Specificity mean: {mean_SP}, median: {med_SP}, std: {std_SP}')
    
    print()

At threshold of 5.0%
MAE mean: 0.039, median: 0.038, std: 0.005
Sensitivity mean: 0.826, median: 0.843, std: 0.058
Specificity mean: 1.0, median: 1.0, std: 0.0

At threshold of 3.0%
MAE mean: 0.039, median: 0.038, std: 0.005
Sensitivity mean: 0.824, median: 0.841, std: 0.066
Specificity mean: 0.989, median: 1.0, std: 0.017

At threshold of 2.0%
MAE mean: 0.039, median: 0.038, std: 0.005
Sensitivity mean: 0.83, median: 0.858, std: 0.068
Specificity mean: 0.97, median: 0.963, std: 0.015

At threshold of 1.0%
MAE mean: 0.039, median: 0.038, std: 0.005
Sensitivity mean: 0.844, median: 0.876, std: 0.074
Specificity mean: 0.948, median: 0.963, std: 0.044

At threshold of 0.5%
MAE mean: 0.039, median: 0.038, std: 0.005
Sensitivity mean: 0.852, median: 0.879, std: 0.07
Specificity mean: 0.9, median: 0.926, std: 0.07

At threshold of 0.1%
MAE mean: 0.039, median: 0.038, std: 0.005
Sensitivity mean: 0.829, median: 0.842, std: 0.07
Specificity mean: 0.793, median: 0.815, std: 0.117

